In [1]:
import pandas as pd
import altair as alt
import os

In [2]:
# read nintendo software and hardware sales data
processed_data_path = os.path.join("..", "data", "processed")
files = sorted([f for f in os.listdir(processed_data_path) if "nintendo" in f])
files

['nintendo_hardware_totals_by_date.csv',
 'nintendo_software_totals_by_date.csv']

In [3]:
hardware = pd.read_csv(os.path.join(processed_data_path, files[0]))
software = pd.read_csv(os.path.join(processed_data_path, files[1]))


In [4]:
set(software["Group"])

{'3DS', 'DS', 'Gamecube', 'Switch', 'Wii', 'Wii U'}

In [5]:
set(hardware["Group"])

{'2DS',
 '2DSXL',
 '3DS',
 '3DSXL',
 'DS',
 'DS Lite',
 'DSi',
 'DSi XL',
 'Gamecube',
 'Lite',
 'OLED',
 'Standard',
 'Wii',
 'Wii U'}

In [6]:
# map hardware to software group
hardware_group_map = {
    '2DS': '2DS/3DS',
    '2DSXL': '2DS/3DS',
    '3DS': '2DS/3DS',
    '3DSXL': '2DS/3DS',
    'DS Lite': 'DS',
    'DSi': 'DS',
    'DSi XL': 'DS',
    'Lite': 'Switch',
    'OLED': 'Switch',
    'Standard': 'Switch',
}

software_group_map = {'3DS': '2DS/3DS'}

software["Group"] = software["Group"].replace(software_group_map)	
software = software.groupby(["Group", "Date"]).sum().reset_index()

hardware["Group"] = hardware["Group"].replace(hardware_group_map)
hardware = hardware.groupby(["Group", "Date"]).sum().reset_index()

In [7]:
software, hardware

(      Group        Date    Qty
 0   2DS/3DS  2011-03-31   9.43
 1   2DS/3DS  2012-03-31  35.99
 2   2DS/3DS  2013-03-31  49.61
 3   2DS/3DS  2014-03-31  67.89
 4   2DS/3DS  2015-03-31  62.75
 ..      ...         ...    ...
 75    Wii U  2019-03-31   0.85
 76    Wii U  2020-03-31   0.25
 77    Wii U  2021-03-31   0.25
 78    Wii U  2022-03-31   0.13
 79    Wii U  2023-03-31   0.12
 
 [80 rows x 3 columns],
       Group        Date    Qty
 0   2DS/3DS  2011-03-31   3.61
 1   2DS/3DS  2012-03-31  13.52
 2   2DS/3DS  2013-03-31  13.96
 3   2DS/3DS  2014-03-31  12.24
 4   2DS/3DS  2015-03-31   8.74
 ..      ...         ...    ...
 75    Wii U  2019-03-31   0.00
 76    Wii U  2020-03-31   0.00
 77    Wii U  2021-03-31   0.00
 78    Wii U  2022-03-31   0.00
 79    Wii U  2023-03-31   0.00
 
 [80 rows x 3 columns])

In [8]:
# join on Group and Date
df = pd.merge(software, hardware, on=["Group", "Date"], how="outer", suffixes=("_software", "_hardware")).sort_values(["Group", "Date"])
df

,Group,Date,Qty_software,Qty_hardware
0,2DS/3DS,2011-03-31,9.43,3.61
1,2DS/3DS,2012-03-31,35.99,13.52
2,2DS/3DS,2013-03-31,49.61,13.96
3,2DS/3DS,2014-03-31,67.89,12.24
4,2DS/3DS,2015-03-31,62.75,8.74
...,...,...,...,...
75,Wii U,2019-03-31,0.85,0.00
76,Wii U,2020-03-31,0.25,0.00
77,Wii U,2021-03-31,0.25,0.00
78,Wii U,2022-03-31,0.13,0.00


In [35]:
selection = alt.selection_point(fields=['Group'], bind='legend')
opacity = alt.condition(selection, alt.value(1), alt.value(0.2))


hw_chart = alt.Chart(df).mark_line().encode(
    x=alt.X('Date:T').axis(title=None),
    y=alt.Y('Qty_hardware:Q').title('Hardware Units (Millions)'),
    color=alt.Color('Group:N', legend=alt.Legend(title="Product Group", labelFontSize=12), scale=alt.Scale(scheme='tableau20')),
    opacity=opacity
)

sw_chart = alt.Chart(df).mark_line(strokeDash=[5,5]).encode(
    x=alt.X('Date:T').axis(title=None),
    y=alt.Y('Qty_software:Q').title('Software Units (Millions)'),
    color=alt.Color('Group:N', legend=alt.Legend(title="Product Group", labelFontSize=12), scale=alt.Scale(scheme='tableau20')),
    opacity=opacity
)

chart = hw_chart + sw_chart
chart = chart.add_params(selection)
chart = chart.resolve_scale(y='independent')
chart = chart.properties(title="Nintendo Hardware and Software Sales (Millions)", width=600, height=550)
chart
# alt.layer(hw_chart, sw_chart)

alt.LayerChart(...)

In [36]:
chart.save("../figs/Nintendo_Software_by_models.png")